In [ ]:
import numpy as np
from sklearn.model_selection import KFold
import random
import os
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
import torchvision
from torchvision import datasets,transforms,models
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

from cnn_finetune import make_model
from sklearn.preprocessing import label_binarize
from itertools import cycle
import seaborn as sns

from sklearn.metrics import auc
from efficientnet_pytorch import EfficientNet

os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
#GPU指定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# EfficientNet-b0 model のロード
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
#ファインチューニング（全てのパラメータを訓練可能に）
for param in model.parameters():
    param.requires_grad = True  
model = model.to(device)

#モデルの構造確認
print(model)

In [ ]:
#データセットの設定
train_dataset = torchvision.datasets.ImageFolder(root='/home/yamaguchi/最終/Ki67値クラス分類データセット/train')  
test_dataset  = torchvision.datasets.ImageFolder(root='/home/yamaguchi/最終/Ki67値クラス分類データセット/test') 


#学習用データの前処理
transform = torchvision.transforms.Compose([
    #ランダムな領域を切り取って画像をリサイズ
    transforms.RandomCrop((224,224)),
    # ランダムに画像を水平方向に反転
    transforms.RandomHorizontalFlip(),
    # ランダムに画像の色調（明るさ，コントラスト）を変更
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    # グレースケールに変換（3チャンネル出力）
    transforms.Grayscale(num_output_channels=3),
    torchvision.transforms.ToTensor(),
    #ピクセル値の正規化
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


#テスト用データの前処理
transform_2 = torchvision.transforms.Compose([
    transforms.RandomCrop((224,224)),
    transforms.Grayscale(num_output_channels=3),
    torchvision.transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#バッチサイズの設定(2の乗数が一般的)
batch_size = 32

train_dataset.transform=transform
test_dataset.transform=transform_2
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False)

#フォルダのクラス名の表示
print(train_dataset.classes)

In [ ]:
#モデルのパラメータ数などの確認
from torchsummary import summary

summary(model, (3, 224, 224))

#https://discuss.pytorch.org/t/what-is-1-in-output-shape-of-a-model-in-torch-summary/67790

In [ ]:
def train(device, model, optimizer, criterion, cv_train_dataloader, cv_valid_dataloader):
    # Early stoppingのための初期設定
    the_last_loss = 100  # 監視する損失の初期値
    patience = 10  # 許容できる損失の増加（非改善）のエポック数
    trigger_times = 0  # 現在の非改善のエポック数

    for epoch in range(100):  # 最大エポック数
        model.train()  # モデルを訓練モードに設定
        running_loss = 0.0  # 現在のエポックの累積損失
        correct_num = 0  # 正しく予測されたデータの数
        total_num = 0  # 総データ数
        batch_count = 0  # 処理したバッチの数

        for data, target in cv_train_dataloader:  # 訓練データのバッチごとのループ
            inputs, labels = data.to(device), target.to(device)  # データをデバイスに移動

            optimizer.zero_grad()  # 勾配をゼロに初期化

            outputs = model(inputs)  # モデルによる予測
            
            loss = criterion(outputs, labels)  # 損失の計算
            predicted = torch.max(outputs.data, 1)[1]  # 予測ラベル
            correct_num_temp = (predicted == labels).sum()  # 正解数の計算
            correct_num += correct_num_temp.item()
            total_num += data.shape[0]
            loss.backward()  # 損失に基づく勾配の計算
            optimizer.step()  # パラメータの更新
            running_loss += loss.item()
            batch_count += 1 

        # エポックごとの損失と正解率の表示
        print('epoch:%d loss: %.3f acc: %.3f' %
             (epoch + 1, running_loss / batch_count, correct_num * 100 / total_num))
            

        # 検証セットでの損失の計算（Early Stoppingのため）
        the_current_loss = validation(model, device, cv_valid_dataloader, criterion)
        print('The current loss:', the_current_loss)

        # Early Stoppingの条件判定
        if the_current_loss > the_last_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)

            if trigger_times >= patience:  # patience回連続で損失が改善しなかった場合
                print('Early stopping!\nStart to test process.')
                return model

        else:
            print('trigger times: 0')
            trigger_times = 0  # 損失が改善された場合、カウンタをリセット

        the_last_loss = the_current_loss  # 監視する損失の更新

    return model

In [ ]:
#検証関数の定義


def validation(model, device, cv_valid_dataloader, criterion):
    model.eval()
    running_loss = 0

    with torch.no_grad():
        for data,target in cv_valid_dataloader:
            inputs, labels = data.to(device), target.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

    return running_loss / len(cv_valid_dataloader)

In [ ]:
# テスト関数の定義
def test(device, model, test_dataloader):
    model.eval()  # モデルを評価モードに設定

    correct_num = 0  # 正解数
    total_num = 0    # 総サンプル数
    predicts_list = []  # 予測結果を格納するリスト
    labels_list = []    # 実際のラベルを格納するリスト
    scores_list = []    # 各クラスの確率を格納するリスト

    with torch.no_grad():  # 勾配計算を無効化
        for data, target in test_dataloader:
            inputs, labels = data.to(device), target.to(device)  # データをデバイスに移動

            outputs = model(inputs)  # モデルで予測
            m = nn.Softmax(dim=1)  # Softmaxを適用して確率を計算
            probs = m(outputs)

            _, predicted = torch.max(outputs.data, 1)  # 最も高い確率を持つクラスを予測値とする
            correct_num_temp = (predicted == labels).sum()  # 正解数を計算
            correct_num += correct_num_temp.item()
            total_num += data.shape[0]

            # CPUにデータを移動してnumpy配列に変換
            labels = labels.to('cpu').numpy()
            predicted = predicted.to('cpu').numpy()
            probs = probs.to('cpu').numpy()

            # リストに追加
            labels_list.append(labels)
            predicts_list.append(predicted)
            scores_list.append(probs)

    # リストをnumpy配列に変換
    labels = np.concatenate(labels_list)
    predicted = np.concatenate(predicts_list)
    scores = np.concatenate(scores_list)

    # ROC曲線とAUCの計算
    labels_bin = label_binarize(labels, classes=[0, 1, 2])  # ラベルを二値化
    fpr, tpr, roc_auc = {}, {}, {}
    for i in range(3):
        fpr[i], tpr[i], _ = roc_curve(labels_bin[:, i], scores[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # 混同行列とその他の指標を計算
    C = confusion_matrix(labels, predicted)
    ac = accuracy_score(labels, predicted)
    pre = precision_score(labels, predicted, average='macro')
    re = recall_score(labels, predicted, average='macro')
    f1 = f1_score(labels, predicted, average='macro')
    AUC = roc_auc_score(labels, scores, multi_class='ovr')  # AUCを計算

    # 結果を表示
    print(C)
    print("\n")
    print("test accuracy : %.3f" % ac)
    print("test precision : %.3f" % pre)
    print("test recall : %.3f" % re)
    print("test F1 : %.3f" % f1)
    print("AUC : %.3f" % AUC)

    for i in range(3):
        print("AUC for class {}: {:.3f}".format(i, roc_auc[i]))

    # マクロ平均ROC曲線とAUCの計算とプロット
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(3)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(3):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= 3

    fpr_macro, tpr_macro, roc_auc_macro = all_fpr, mean_tpr, auc(all_fpr, mean_tpr)
    plt.figure()
    plt.plot(fpr_macro, tpr_macro, label='macro-average ROC curve (area = {:.3g})'.format(roc_auc_macro),
             color='navy', linestyle=':', linewidth=4)

    # クラスごとのROC曲線をプロット
    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    class_labels = ['class 0', 'class 1', 'class 2']
    for i, color in zip(range(3), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2, label='ROC curve of class {0} (area = {1:.2f})'.format(class_labels[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.show()

    # 混同行列をプロット
    plt.figure(figsize=(8, 6))
    sns.heatmap(C, annot=True, cmap='Blues', fmt='g', xticklabels=class_labels, yticklabels=class_labels, annot_kws={'size': 20})
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
#k分割交差検証

kf = KFold(n_splits=10)

In [ ]:
#プログラムの実行時間の算出
%%time


#main関数
def main():
    
    for _fold, (train_index, valid_index) in enumerate(kf.split(np.arange(len(train_dataset)))):
    
        #交差検証に伴う使用モデルの再定義
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=3)
       
        for param in model.parameters():
            param.requires_grad = True  
        model = model.to(device)


        batch_size = 32
        #損失関数の定義
        criterion = nn.CrossEntropyLoss()
        #最適化関数の定義
        optimizer = optim.SGD(model.parameters(), lr=0.001,momentum=0.9)
        
        #交差検証に伴うデータセットの分割
        cv_train_dataset = Subset(train_dataset, train_index)
        cv_train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
        cv_valid_dataset   = Subset(train_dataset, valid_index)
        cv_valid_dataloader = DataLoader(train_dataset, batch_size, shuffle=False)
        
        print('Fold {}------------------------------------------------------------------------------'.format(_fold+1))

        model = train(device, model, optimizer, criterion, cv_train_dataloader, cv_valid_dataloader)
        #パラメータの保存
        torch.save(model.state_dict(), 'sample_' + str(_fold) + '.pt')
        #test関数の実行
        test(device, model, test_dataloader)


if __name__ == '__main__':
    main()